# Lab | Data Aggregation and Filtering

In this challenge, we will continue to work with customer data from an insurance company. We will use the dataset called marketing_customer_analysis.csv, which can be found at the following link:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis.csv

This dataset contains information such as customer demographics, policy details, vehicle information, and the customer's response to the last marketing campaign. Our goal is to explore and analyze this data by first performing data cleaning, formatting, and structuring.

In [1]:
#import libraries
import pandas as pd
#load the data
marketing_customer_analysis = pd.read_csv("https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis_clean.csv")

# cleaning up the data:
# * the unnamed:_0 column which looks like an old index and should be removed. 
marketing_customer_analysis = marketing_customer_analysis.drop(columns=["unnamed:_0"])
# * one of the column names (employmentstatus)
marketing_customer_analysis.rename(columns={"employmentstatus":"employment_status"}, inplace=True)
# * number_of_open_complaints which should be an int 
marketing_customer_analysis.number_of_open_complaints = marketing_customer_analysis.number_of_open_complaints.apply(lambda x: round(x)).astype(int)
# * months_since_last_claim which should be an int.
marketing_customer_analysis.months_since_last_claim = marketing_customer_analysis.months_since_last_claim.apply(lambda x: round(x)).astype(int)
# * total_claim_amount could be standardised to 2 decimal places. 
marketing_customer_analysis.total_claim_amount = marketing_customer_analysis.total_claim_amount.apply(lambda x: round(x, 2))

1. Create a new DataFrame that only includes customers who have a total_claim_amount greater than $1,000 and have a response of "Yes" to the last marketing campaign.

In [16]:
high_claims = marketing_customer_analysis[(marketing_customer_analysis.response == "Yes") & (marketing_customer_analysis.total_claim_amount > 1000)]

2. Using the original Dataframe, analyze the average total_claim_amount by each policy type and gender for customers who have responded "Yes" to the last marketing campaign. Write your conclusions.

In [17]:
marketing_customer_analysis[marketing_customer_analysis.response == "Yes"].groupby(["policy_type", "gender"]).total_claim_amount.mean()

# The higher average total claim amount corresponds to Personal Auto policies. In corporate auto, the average total claim amount is lower. 

policy_type     gender
Corporate Auto  F         433.738462
                M         408.582468
Personal Auto   F         452.966037
                M         457.010149
Special Auto    F         453.279714
                M         429.528125
Name: total_claim_amount, dtype: float64

3. Analyze the total number of customers who have policies in each state, and then filter the results to only include states where there are more than 500 customers.

In [21]:
customers_by_state = marketing_customer_analysis.groupby(["state"]).customer.nunique()
customers_by_state[customers_by_state > 500]

state
Arizona       1703
California    3520
Nevada         882
Oregon        2601
Washington     798
Name: customer, dtype: int64

4. Find the maximum, minimum, and median customer lifetime value by education level and gender. Write your conclusions.

In [23]:
marketing_customer_analysis.groupby(["education", "gender"]).customer_lifetime_value.agg(["mean", "max", "min"])

mean          max          min
education            gender                                       
Bachelor             F       7874.269478  73225.95652  1904.000852
                     M       7703.601675  67907.27050  1898.007675
College              F       7748.823325  61850.18803  1898.683686
                     M       8052.459288  61134.68307  1918.119700
Doctor               F       7328.508916  44856.11397  2395.570000
                     M       7415.333638  32677.34284  2267.604038
High School or Below F       8675.220201  55277.44589  2144.921535
                     M       8149.687783  83325.38119  1940.981221
Master               F       8157.053154  51016.06704  2417.777032
                     M       8168.832659  50568.25912  2272.307310

## Bonus

5. The marketing team wants to analyze the number of policies sold by state and month. Present the data in a table where the months are arranged as columns and the states are arranged as rows.

6.  Display a new DataFrame that contains the number of policies sold by month, by state, for the top 3 states with the highest number of policies sold.

*Hint:*
- *To accomplish this, you will first need to group the data by state and month, then count the number of policies sold for each group. Afterwards, you will need to sort the data by the count of policies sold in descending order.*
- *Next, you will select the top 3 states with the highest number of policies sold.*
- *Finally, you will create a new DataFrame that contains the number of policies sold by month for each of the top 3 states.*

7. The marketing team wants to analyze the effect of different marketing channels on the customer response rate.

Hint: You can use melt to unpivot the data and create a table that shows the customer response rate (those who responded "Yes") by marketing channel.

External Resources for Data Filtering: https://towardsdatascience.com/filtering-data-frames-in-pandas-b570b1f834b9

In [ ]:
# your code goes here